# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

# 📙Web Scraping Solution

## 🟠Step 1: Install dependencies as per requirements file

In [15]:
!python -m pip install -r ../requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [16]:
#import os
#from bs4 import BeautifulSoup  # De la libreria bs4 (Nuevo nombere de la libreria) traeme la funcion BeautifulSoup
#import requests
#import time
#import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

## 🟠Step 2: Download HTML
The download of the HTML of the web page will be done with the requests library, as we saw in the module theory.

The web page we want to scrape is the following: https://ycharts.com/companies/TSLA/revenues. It collects and stores information about the growth of the company every three months, since June 2009. It stores the text scraped from the web in some variable.

In [17]:
import os # Importo libreria necesaria

# Para evitar que la respuesta sea "403 Forbidden", intento con un User Anonimo. (If no information is extracted, then connect as anonymous).
# Para esto defino un User-Agent falso para que la solicitud parezca provenir de un navegador real, ya que
# los servidores bloquean a los bots, pero permiten navegadores con User-Agent válidos.

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"}

# EXPLICACION DEL USER-AGENT

# 1. headers
#     headers es una variable que contiene un diccionario. En este caso, se usa para definir un conjunto de cabeceras HTTP (headers) que se enviarán junto con la solicitud HTTP.

# 2. ¿Qué son los "headers" en una solicitud HTTP?
#     Headers (cabeceras) son información adicional que se envía junto con una solicitud HTTP para proporcionar detalles sobre la solicitud o sobre el cliente que la está haciendo (por ejemplo, el navegador que estás usando).

# 3. El header User-Agent
#     En este caso, estás utilizando un header específico llamado User-Agent.
#     El User-Agent le dice al servidor qué tipo de cliente está haciendo la solicitud (por ejemplo, qué navegador web está usando).

# 4. El valor del User-Agent
#     El valor de este User-Agent es:
#     "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"
#     Este es un string largo que indica varias cosas sobre el navegador y el sistema operativo de la persona que está haciendo la solicitud. Aquí te desgloso cada parte:
#
#     Mozilla/5.0: 
#       Indica que el navegador es compatible con el estándar Mozilla (aunque ahora muchos navegadores usan este mismo formato, como Chrome y Safari). Esto es histórico y es parte del proceso de compatibilidad entre navegadores.
#     (Macintosh; Intel Mac OS X 10_11_5): 
#       Indica que el cliente está utilizando un sistema operativo Mac OS X en una máquina con un procesador Intel.
#     AppleWebKit/537.36: 
#       Esto hace referencia al motor de renderizado que utiliza el navegador. AppleWebKit es el motor que usa Safari y Chrome.
#     (KHTML, like Gecko): 
#       Es una declaración de compatibilidad con el motor de renderizado KHTML (usado en navegadores como Konqueror) y Gecko (usado en Firefox). Es una forma de asegurar que el servidor pueda interpretar correctamente las solicitudes de diferentes navegadores.
#     Chrome/50.0.2661.102: 
#       Informa que el navegador es Chrome versión 50.0.2661.102.
#     Safari/537.36: 
#       Esto indica que el navegador también es compatible con Safari, versión 537.36.

# 5. ¿Por qué usar un User-Agent falso?
#     Simular un navegador real: Algunos sitios web bloquean solicitudes automáticas (como las de bots o scripts) si detectan que vienen de una fuente que no parece un navegador estándar. Al configurar un User-Agent como el que has puesto, haces que tu solicitud parezca provenir de un navegador real (en este caso, un navegador Chrome en un Mac), lo que puede ayudar a evitar bloqueos por parte del servidor.

# 6. ¿Cómo se usa en el código?
#     Cuando usas este User-Agent en la solicitud, lo estás añadiendo a las cabeceras de la solicitud HTTP. Así, el servidor recibirá la solicitud con el mismo User-Agent que usaría un navegador real:
#       response = requests.get(url, headers=headers)
#     Aquí estás enviando el User-Agent dentro de las cabeceras HTTP de la solicitud, lo que hace que la solicitud parezca más legítima y difícil de bloquear.

# 7. En resumen:
#    El User-Agent es una cadena de texto que identifica el navegador y el sistema operativo que está haciendo la solicitud. Usarlo en las cabeceras de una solicitud HTTP puede hacer que el servidor piense que la solicitud proviene de un navegador real, lo que puede ayudarte a evitar bloqueos al hacer web scraping o solicitudes automáticas a ciertos sitios.

In [18]:
# Importo librerias necesarias:
import requests # Generalmente no usamos alias
import time # Generalmente no usamos alias

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" # Defino la url de la que voy a extraer informacion
response = requests.get(url, headers=headers) # Hace una nueva solicitud a la url definida, con el User-Agent para intentar evitar el bloqueo.
time.sleep(10) # Pausa el código por 10 segundos, posiblemente para evitar ser detectado como bot.
response # Verifico si la respuestra a la solicitud http es valida (Por ejemplo, 200 en vez de 403).

<Response [200]>

In [19]:
# Guardo el codigo en un csv para analizarlo en caso de necesitar.
if response: # Si existe codigo
    with open ("codigo_tesla.csv","wb") as file: 
        file.write(response.content)


# with open es la funcion para guardado. Entre parentesis ponemos la ruta donde guardar el 
    # archivo y el nombre que le daremos.
    # luego de la coma, le indicamos el metodo de escritura o apertura. (wb = write binary)
    # En este caso es escritura.
    # luego le decimos el alias
    # Es decir, con la funcion with open podemos abrir un archivo para lo que uno necesite, ya sea
    # lectura o escritura. 
    # El with nos asegura de que el archivo una vez que se abre, se cierre. Si pongo solo el open, 
    # luego tendremos que hacer el close. 
    # Veremos que dentro de la carpeta data / raw se abra generado el archivo csv (coma separeted values).
    
    # "w": Escritura (write mode), abre el archivo para escribir en el. Si el archivo ya existe, se sobrescribira. Si no existe, se creara.
    # "b": Modo Binario (binary mode), se puede combinar con otros modos ("rb, "wb", "ab") para trabajar con archivos binarios.

In [20]:
html_data = response.text # Guarda el HTML en html_data
html_data # Muestro el HTML obtenido para verificar que se este logrando el cometido

'\r\n<!DOCTYPE html>\r\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\r\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\r\n<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->\r\n<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->\r\n    <head>\r\n        <meta charset="utf-8">\r\n        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\r\n\t\t<link rel="canonical" href="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" />\r\n\t\t<link rel="alternate" media="only screen and (max-width: 768px)" href="https://m.macrotrends.net/stocks/charts/TSLA/tesla/revenue">\r\n\t\t<title>Tesla Revenue 2010-2024 | TSLA | MacroTrends</title>\r\n\t\t<link rel="icon" type="image/x-icon" href="/assets/images/icons/FAVICON/macrotrends-favicon-96.png">\r\n\t\t<meta name="description" content="Tesla annual/quarterly revenue history and growth rate from 2010 to 2024. Revenue can be defined as the amount of mo

## 🟠Step 3: Transform the HTML
The next step to start extracting the information is to transform it into a structured object. Do this using BeautifulSoup. Once you have interpreted the HTML correctly, parse it to:

1. Find all the tables.
2. Find the table with the quarterly evolution.
3. Store the data in a DataFrame.


In [21]:
# Transform the extraction into a structured object using BeautifulSoup

from bs4 import BeautifulSoup # De la libreria bs4 (Nuevo nombere de la libreria) traeme la funcion BeautifulSoup

soup = BeautifulSoup(html_data,"html.parser") # usa la librería BeautifulSoup para analizar el código HTML y convertirlo en un objeto que puedes manipular fácilmente con Python.
soup


<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" rel="canonical"/>
<link href="https://m.macrotrends.net/stocks/charts/TSLA/tesla/revenue" media="only screen and (max-width: 768px)" rel="alternate"/>
<title>Tesla Revenue 2010-2024 | TSLA | MacroTrends</title>
<link href="/assets/images/icons/FAVICON/macrotrends-favicon-96.png" rel="icon" type="image/x-icon"/>
<meta content="Tesla annual/quarterly revenue history and growth rate from 2010 to 2024. Revenue can be defined as the amount of money a company receives from its customers in exchange for the sales of goods or services.  Reve

### 🧩3.1 Find all the tables

In [22]:
# Find all tables
tables = soup.find_all("table") # soup es un objeto de BeautifulSoup que contiene el HTML de la página. find_all("table") busca y devuelve todas las etiquetas <table> dentro del HTML. El resultado es un ResultSet, que es una lista de elementos BeautifulSoup. 
tables # tables es entonces una lista de objetos BeautifulSoup, cada uno representando una tabla del HTML. Esto indica que tables contiene todas las tablas encontradas en el HTML.

# 💛 OTROS EJEMPLOS DE COMO USAR LOS DATOS EXTRAIDOS 💛:
# soup.find_all("table")            : Encuentra todas las tablas en el HTML
# tables[0]                         : Obtiene la primera tabla de la lista
# tables[0].get_text()	            : Extrae solo el texto de la primera tabla
# pd.read_html(str(tables[0]))[0]   : Convierte la primera tabla en un DataFrame, importando previamente la libreria pandas

# 💛 ESTRUCTURA DE LAS TABLAS EN EL CODIGO 💛:
# <th> = headers de la tabla
# <tr> = Rows de la tabla 
# <Caption> = Titulo de la tabla
# <td> = Celda de la tabla
# Por ahora vemos que la info es legible. 
# Tener en cuenta que todo lo que trae lo interpreta en formato texto, 
# asi que los numeros como % los tendremos que convertir a su formato Correcto.

[<table class="historical_data_table table">
 <thead>
 <tr>
 <th colspan="2" style="text-align:center">Tesla Annual Revenue<br/><span style="font-size:14px;">(Millions of US $)</span></th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td style="text-align:center">2023</td>
 <td style="text-align:center">$96,773</td>
 </tr>
 <tr>
 <td style="text-align:center">2022</td>
 <td style="text-align:center">$81,462</td>
 </tr>
 <tr>
 <td style="text-align:center">2021</td>
 <td style="text-align:center">$53,823</td>
 </tr>
 <tr>
 <td style="text-align:center">2020</td>
 <td style="text-align:center">$31,536</td>
 </tr>
 <tr>
 <td style="text-align:center">2019</td>
 <td style="text-align:center">$24,578</td>
 </tr>
 <tr>
 <td style="text-align:center">2018</td>
 <td style="text-align:center">$21,461</td>
 </tr>
 <tr>
 <td style="text-align:center">2017</td>
 <td style="text-align:center">$11,759</td>
 </tr>
 <tr>
 <td style="text-align:center">2016</td>
 <td style="text-align:center">$7,000</td>
 </tr>
 <

In [23]:
# 💛 TIP IMPORTANTE! 💛:
# Esta buena copiar esto a un editor como notepad+, 
# para identificar ahi los distintos elementos de las tablas, la cantidad de tablas, etc.
# Si luego pego desde el notepad+ a excel, ya me muestra las tablas con formato, para visualizarlas mejor aun.

### 🧩3.2 Find the table with the quarterly evolution

In [34]:
import pandas as pd  


# 🧩 Paso 2:
# Encontrar la tabla correcta
for index, table in enumerate(tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        table_index = index
        break
print(table_index)

tesla_revenue = pd.DataFrame(columns = ["Date", "Revenue"])
tesla_revenue

1


,Date,Revenue


In [27]:
# 🧩 Paso 1: 
# Importa la librería Pandas, que es clave para manipular y analizar datos en Python.
import pandas as pd  


# 🧩 Paso 2:
# Encontrar la tabla correcta
for index, table in enumerate(tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        table_index = index
        break

# Explicacion:
    # tables es la lista de tablas extraídas (con beautifulsoup) de la url definida, 
    # Se recorre tables con enumerate(), que da dos valores:
    #     1. index: la posición de la tabla en la lista.
    #     2. table: la tabla en sí.

    # Se convierte cada table en texto con str(table).
    # Se busca una tabla que contenga la frase "Tesla Quarterly Revenue".
    # Cuando la encuentra, guarda su índice en table_index y detiene el bucle con break.


# 🧩 Paso 3: 
# Crear un DataFrame vacío con dos columnas: "Date" y "Revenue", donde se guardarán los datos extraídos.
tesla_revenue = pd.DataFrame(columns = ["Date", "Revenue"])


# 🧩 Paso 4: 
# Extraer datos de la tabla

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Date = col[0].text
        Revenue = col[1].text.replace("$", "").replace(",", "")
        tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({
            "Date": Date,
            "Revenue": Revenue
        }, index = [0])], ignore_index = True)

# Explicacion:
#  (1) Recorre todas las filas (tr) dentro del tbody de la tabla seleccionada (tables[table_index]).
#  (2) Para cada fila (row), encuentra las celdas (td) con find_all("td").
#  (3) Si col no está vacío (col != []):
#       (*)Extrae la fecha (Date) de la primera celda (col[0].text).
#       (*)Extrae el revenue (Revenue) de la segunda celda (col[1].text), quitando "$" y "," con .replace().

#  (4) Agrega los datos al DataFrame usando pd.concat(), creando una nueva 
#      fila como DataFrame y uniéndola a tesla_revenue.

#  (5) ignore_index=True evita problemas con los índices.


# 🧩 Paso 5: 
# Mostrar los primeros datos
tesla_revenue.head()


# 🧩 Resumen Final:
#     (1) Busca la tabla correcta dentro de una lista de tablas.
#     (2) Crea un DataFrame vacío con columnas Date y Revenue.
#     (3) Extrae las filas de la tabla y limpia los valores.
#     (4) Agrega las filas al DataFrame.
#     (5) Muestra los primeros resultados.

,Date,Revenue
0,2024-09-30,25182
1,2024-06-30,25500
2,2024-03-31,21301
3,2023-12-31,25167
4,2023-09-30,23350
